In [1]:
%%HTML
<style type='text/css'>
    *{
        # background-color:#E3EDCD;
        # background-color:black;
        # color:white;
        
    }
    h1{
        color:#1976d2;
    }
    h2{
        color:#f57c00;
    }
    h3{
        color:#ba37ff;
    }
    h4{
        color:green;
    }
    table{
        border:1px solid black !important;
        border-collapse:collapse !important;
    }
    th{
        background-color:blueviolet !important;
        text-align:center;
        color:white;
    }
    th,td{
        border:0.1px solid black !important;
        transition:0.2s all liner;
        
    }
    td:hover{
        transform:scale(1.1);
        background-color:orange;
        color:blueviolet;
    }
    .raw{
        white-space:pre;
    }
    .atcenter{
        text-align:center !important;
    }
    #imp{
        color:red;
        font-weight:bolder;
    }
</style>

# 双十一销售数据可视化分析

本数据集收集27598条数据  
- update_time: 更新时间
- id:          编号
- title:        商品标题
- price:价格
- sale_count:销售量
- comment_count:评论数量
- 店名:店铺名称

## 导入要用到的包

In [2]:
import pandas as pd
import numpy as np
import pyecharts.options as opts
from pyecharts.globals import CurrentConfig,NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts.charts import Pie,Line,Bar,Map,Timeline

## 数据准备

In [3]:
data = pd.read_csv('../others/double11beautysales.csv')
data.head()

,update_time,id,title,price,sale_count,comment_count,店名
0,2022/11/14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂
1,2022/11/14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂
2,2022/11/14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂
3,2022/11/14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂
4,2022/11/14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598 entries, 0 to 27597
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   update_time    27598 non-null  object 
 1   id             27598 non-null  object 
 2   title          27598 non-null  object 
 3   price          27598 non-null  float64
 4   sale_count     25244 non-null  float64
 5   comment_count  25244 non-null  float64
 6   店名             27598 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.5+ MB


In [5]:
data.columns

Index(['update_time', 'id', 'title', 'price', 'sale_count', 'comment_count',
       '店名'],
      dtype='object')

### 空值统计

In [6]:
data.isnull().sum()

update_time         0
id                  0
title               0
price               0
sale_count       2354
comment_count    2354
店名                  0
dtype: int64

In [7]:
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
27593    False
27594    False
27595    False
27596    False
27597    False
Length: 27598, dtype: bool

### 重复值处理

#### 查看是否有重复值

In [8]:
data.duplicated().sum()

np.int64(86)

可以看到有86条数据是重复的

#### 去除重复值

subset参数用来设置以哪些列的重复作为重复的标准，参数为列标签，如果不设置该值，则默认为以所有列作为重复的判断条件。

```python
例如:
print(data.drop_duplicates(['id', 'name']))

```

keep可以设置为三个参数，默认为first
first表示保留第一次出现的记录
last表示保留最后一次出现的记录
False表示把所有重复的删除

inplace可以设置为True或False，默认为False  
True表示原地去重，会改变dataframe  
False表示会返回一个新的dataframe，不会改变原来的变量  

In [9]:
data.drop_duplicates(keep='last',inplace=True) 
#这样不用新的对象来接收,因为我们指定了inplace=True,因为我们直接对原数据进行操作并且去重保留最后一个,即取出重复数据中的一个作为不重复的

<p id='imp'>注意,去除重复值之后索引也发生了变化,所以我们要重新编序索引</p>

In [10]:
data.reset_index(drop=True,inplace=True) #重置索引

In [11]:
data.head()

,update_time,id,title,price,sale_count,comment_count,店名
0,2022/11/14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂
1,2022/11/14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂
2,2022/11/14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂
3,2022/11/14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂
4,2022/11/14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27512 entries, 0 to 27511
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   update_time    27512 non-null  object 
 1   id             27512 non-null  object 
 2   title          27512 non-null  object 
 3   price          27512 non-null  float64
 4   sale_count     25162 non-null  float64
 5   comment_count  25162 non-null  float64
 6   店名             27512 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.5+ MB


### 缺失值处理

In [13]:
data.isnull().sum()

update_time         0
id                  0
title               0
price               0
sale_count       2350
comment_count    2350
店名                  0
dtype: int64

In [14]:
data.fillna(0,inplace=True) #采用0填充法进行空值填充

In [15]:
data.isnull().sum()

update_time      0
id               0
title            0
price            0
sale_count       0
comment_count    0
店名               0
dtype: int64

可以看到填充成功,已经没有空值了

## 时间序列分析

In [16]:
data.head()

,update_time,id,title,price,sale_count,comment_count,店名
0,2022/11/14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂
1,2022/11/14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂
2,2022/11/14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂
3,2022/11/14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂
4,2022/11/14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂


#### 将更新时间update_time修改为时间日期类型

In [17]:
data['update_time']

0        2022/11/14
1        2022/11/14
2        2022/11/14
3        2022/11/14
4        2022/11/14
            ...    
27507     2022/11/5
27508     2022/11/5
27509     2022/11/5
27510     2022/11/5
27511     2022/11/5
Name: update_time, Length: 27512, dtype: object

In [18]:
# 先将其转换为时间类
data['update_time']=pd.to_datetime(data['update_time'])

In [19]:
data['update_time']

0       2022-11-14
1       2022-11-14
2       2022-11-14
3       2022-11-14
4       2022-11-14
           ...    
27507   2022-11-05
27508   2022-11-05
27509   2022-11-05
27510   2022-11-05
27511   2022-11-05
Name: update_time, Length: 27512, dtype: datetime64[ns]

In [20]:
data['update_time']=data['update_time'].map(lambda x: x.strftime('%Y-%m-%d'))

In [21]:
data.head()

,update_time,id,title,price,sale_count,comment_count,店名
0,2022-11-14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂
1,2022-11-14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂
2,2022-11-14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂
3,2022-11-14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂
4,2022-11-14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂


### 新增一列数据作为销售额

In [22]:
data['sale_amount'] = data['price']*data['sale_count']

In [23]:
data.head()

,update_time,id,title,price,sale_count,comment_count,店名,sale_amount
0,2022-11-14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂,3713941.0
1,2022-11-14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂,1575668.0
2,2022-11-14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂,1254132.0
3,2022-11-14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂,980590.0
4,2022-11-14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂,722244.0


#### 按照销量来排序

利用掩码操作,选出销量>0的来进行按照销量排序

In [24]:
data[data['sale_count']>0].sort_values(by=['sale_count'],ascending=False)

,update_time,id,title,price,sale_count,comment_count,店名,sale_amount
17026,2022-11-14,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1923160.0,197949.0,妮维雅,80772720.0
17126,2022-11-13,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,37.9,1921582.0,198774.0,妮维雅,72827957.8
17228,2022-11-12,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,37.9,1920083.0,199062.0,妮维雅,72771145.7
17339,2022-11-11,A24304992630,2瓶更划算*妮维雅男士洗面奶控油祛痘印保湿去黑头去油洁面乳护肤,35.0,1886100.0,199532.0,妮维雅,66013500.0
17470,2022-11-10,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1827562.0,200154.0,妮维雅,76757604.0
...,...,...,...,...,...,...,...,...
26512,2022-11-07,A539854888277,Herborist/佰草集御五行焕肌眼霜15g,580.0,1.0,1.0,佰草集,580.0
26511,2022-11-07,A539854255262,Herborist/佰草集御五行焕肌滋养晚霜50g,620.0,1.0,0.0,佰草集,620.0
26530,2022-11-07,A541190557158,Herborist/佰草集新美肌梦幻曲面贴膜3片 保湿补水,1.0,1.0,0.0,佰草集,1.0
4815,2022-11-08,A540638275835,innisfree/悦诗风吟 莹润调色气垫隔离霜限量版 水润保湿修饰隔离,236.0,1.0,0.0,悦诗风吟,236.0


## 进行可视化

### 每日销售量的统计分析

<p id='imp'>注意,这里不能用count了,要是用count得到的是销售个数</p>

In [25]:
salecount_perday = data.groupby('update_time').sum()

In [26]:
salecount_perday

,id,title,price,sale_count,comment_count,店名,sale_amount
update_time,,,,,,,
2022-11-05,A18177226992A18177318911A18190290933A181910171...,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水自然堂 活泉补水洁面...,1223364.48,32021763.0,2978342.0,自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自...,4.485046e+09
2022-11-06,A18177226992A18177318911A18190290933A181910171...,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水自然堂 活泉补水洁面...,1223810.48,32617979.0,3016238.0,自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自...,4.576596e+09
2022-11-07,A18177226992A18177318911A18190290933A181910171...,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水自然堂 活泉补水洁面...,1201752.97,31596576.0,2881135.0,自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自...,4.486687e+09
2022-11-08,A18177226992A18177318911A18190290933A181910171...,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水自然堂 活泉补水洁面...,1188551.58,32666289.0,2978509.0,自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自...,4.621338e+09
2022-11-09,A18177226992A18177318911A18190290933A181910171...,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水自然堂 活泉补水洁面...,1201911.89,33323552.0,2999176.0,自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自...,4.685033e+09
2022-11-10,A18177226992A18177318911A18190290933A181910171...,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水自然堂 活泉补水洁面...,1231314.92,31775729.0,2915972.0,自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自...,4.663642e+09
2022-11-11,A18177226992A18190290933A18191681943A184227978...,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水自然堂 活泉深层净化...,807026.61,27818183.0,2503203.0,自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自...,3.354698e+09
2022-11-12,A18164178225A18177105952A18178033846A181780452...,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜CHANDO/自然堂凝...,649584.51,28821197.0,2586529.0,自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自...,3.746736e+09
2022-11-13,A18164178225A18177105952A18177226992A181780338...,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜CHANDO/自然堂凝...,634479.82,29301688.0,2672284.0,自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自然堂自...,3.768293e+09


In [27]:
salecount_perday['sale_count'] #这样就拿到了每天全部商品销售量

update_time
2022-11-05    32021763.0
2022-11-06    32617979.0
2022-11-07    31596576.0
2022-11-08    32666289.0
2022-11-09    33323552.0
2022-11-10    31775729.0
2022-11-11    27818183.0
2022-11-12    28821197.0
2022-11-13    29301688.0
2022-11-14    29953666.0
Name: sale_count, dtype: float64

#### 转换为字典形式,方便对数据进行处理

In [28]:
result = salecount_perday['sale_count'].to_dict()
result

{'2022-11-05': 32021763.0,
 '2022-11-06': 32617979.0,
 '2022-11-07': 31596576.0,
 '2022-11-08': 32666289.0,
 '2022-11-09': 33323552.0,
 '2022-11-10': 31775729.0,
 '2022-11-11': 27818183.0,
 '2022-11-12': 28821197.0,
 '2022-11-13': 29301688.0,
 '2022-11-14': 29953666.0}

### 作图

In [123]:
c1 = (
    Line()
    .add_xaxis(list(result.keys()))
    .add_yaxis('',list(result.values()),areastyle_opts=opts.AreaStyleOpts(
        opacity=0.7
    ),is_smooth=True)
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title='销售量统计'
        ),
        xaxis_opts=opts.AxisOpts(
            name='日期',
            name_location='end',
            boundary_gap=False,##让面积图从y轴线开始,取消空隙
        ),
        yaxis_opts=opts.AxisOpts(
            name='销售量',
            name_location='end',
            min_=25000000,
            max_=35000000,
            
        ),
        toolbox_opts=opts.ToolboxOpts(),
        visualmap_opts=opts.VisualMapOpts(
            min_=25000000,
            max_=35000000,
            pos_right=0
        )
    )
    .set_series_opts(
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(
                    type_='max',
                    itemstyle_opts=opts.ItemStyleOpts(
                        color='red'
                    )
                ),
                opts.MarkPointItem(
                    type_='min',
                    itemstyle_opts=opts.ItemStyleOpts(
                        color='green'
                    )
                    
                )
            ]
        ),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(
                    type_='average',
                    name='品均值',
                    linestyle_opts=opts.LineStyleOpts(
                        color='green'
                    )
                )
            ],
            label_opts=opts.LabelOpts(
                formatter='{b}:{c}'
            )
        )
    )
)

In [124]:
c1.load_javascript()

In [125]:
c1.render_notebook()